## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import geopandas as gpd


In [27]:
tec = pd.read_parquet("/Rotas-Inteligentes/data/atendTec.parquet")
com = pd.read_parquet("/Rotas-Inteligentes/data/ServCom.parquet")
equipes = pd.read_parquet("/Rotas-Inteligentes/data/equipes.parquet")


In [ ]:
display(tec.shape, com.shape, equipes.shape)


(65367, 22)

(253958, 18)

(31346, 14)

In [ ]:
localidades = gpd.read_file("/Rotas-Inteligentes/shp_localidade/LocalidadesSGD.shp")


In [ ]:
localidades = localidades.set_crs(epsg=31980)


In [ ]:
localidades = localidades.to_crs(epsg=4326)


In [ ]:

localidades = localidades[localidades["CODLOCAL"] == 1]


In [ ]:

localidades


,IDENTIFICA,ID,NAME,SOURCE_ID,SOURCE_DAT,SOURCE_D1,CODLOCAL,CODIBGE,REGIONAL,POLO,SUP,geometry
23,24.0,4328.0,PORTO VELHO,NaN,None,None,1.0,NaN,1.NORTE,PORTO VELHO,None,"POLYGON ((-63.93601 -8.59286, -63.93412 -8.588..."
81,82.0,180351.0,PORTO VELHO_FIC,NaN,None,None,1.0,NaN,1.NORTE,PORTO VELHO,None,"POLYGON ((-63.53473 -7.97433, -63.62078 -7.969..."


In [ ]:
tec["geometry"] = gpd.points_from_xy(tec["LONGITUDE"], tec["LATITUDE"])
tec = gpd.GeoDataFrame(tec, geometry="geometry")
tec = tec.set_crs(epsg=4326)


In [ ]:
base_tec = gpd.sjoin(tec, localidades[["geometry"]], how="inner", predicate='within')


In [ ]:
base_tec.shape


(64606, 24)

In [ ]:
com["geometry"] = gpd.points_from_xy(com["LONGITUDE"], com["LATITUDE"])
com = gpd.GeoDataFrame(com, geometry="geometry")
com = com.set_crs(epsg=4326)


In [ ]:
base_com = gpd.sjoin(com, localidades[["geometry"]], how="inner", predicate='within')


In [ ]:
base_com.shape


(250827, 20)

In [ ]:
base_com = pd.DataFrame(base_com.drop(columns=["geometry", "index_right"]))


In [ ]:
base_tec = pd.DataFrame(base_tec.drop(columns=["geometry", "index_right"]))


In [ ]:
base_tec.to_parquet("/Rotas-Inteligentes/data/atendTec.parquet", index=False, compression="snappy")
base_com.to_parquet("/Rotas-Inteligentes/data/ServCom.parquet", index=False, compression="snappy")


In [ ]:
base_tec.to_csv("/Rotas-Inteligentes/data/atendTec.csv", index=False, sep=";", encoding="utf-8-sig", decimal=",", float_format="%.6f")
base_com.to_csv("/Rotas-Inteligentes/data/ServCom.csv", index=False, sep=";", encoding="utf-8-sig", decimal=",", float_format="%.6f")


In [36]:
frames = []
for file in os.listdir("/Rotas-Inteligentes/results_v3/"):
    if file.endswith(".parquet") and file.startswith("atribuicoes_"):
        df = pd.read_parquet(os.path.join("/Rotas-Inteligentes/results_v3/", file))
        frames.append(df)
df = pd.concat(frames)


In [ ]:
display(
    tec.shape, 
    com.shape, 
    df.shape
)


(65367, 22)

(253958, 18)

(152226, 55)

## Tratamento da base

In [4]:
tec["TIPSERV"] = "T"
tec["DATA_VENC"] = tec["DH_INICIO"]
tec = tec.rename(
    columns={
        "DH_INICIO": "DATA_SOL",
        "DH_ALOCACAO": "DATASAIDA",
        "DH_CHEGADA": "DATAINITRAB",
        "DH_FINAL": "DATATERTRAB",
    }
)


In [5]:
tec = tec.drop_duplicates(subset=["NUMOS"])
com = com.drop_duplicates(subset=["NUMOS"])


In [6]:
com = com[com["DATA_VENC"] > com["DATA_SOL"]]


In [7]:
com = com.rename(columns={"NUM_CDC": "DEFEITO_FALHA"})
com["TIPO_BASE"] = "COMERCIAL"


In [8]:
tec["CODSERV"] = np.nan


In [9]:
com.loc[com["TD"] >= 5]["TD"].describe()


count    106366.000000
mean         19.790451
std          41.889856
min           5.000000
25%           8.830000
50%          14.250000
75%          24.320000
max       11549.600000
Name: TD, dtype: float64

In [10]:
display(com.columns, tec.columns)


Index(['NUMOS', 'TIPSERV', 'CODSERV', 'DATA_SOL', 'DATA_VENC', 'DEFEITO_FALHA',
       'LATITUDE', 'LONGITUDE', 'DATASAIDA', 'DATAINITRAB', 'DATATERTRAB',
       'EQUIPE', 'TD', 'TE', 'ANO', 'MES', 'EUSD_FIO_B', 'EUSD', 'TIPO_BASE'],
      dtype='object')

Index(['TIPO_BASE', 'NUMOS', 'ABRANGENCIA', 'DEFEITO_FALHA', 'LOCALIDADE',
       'LOCALIZACAO', 'NOX', 'NOY', 'DATA_SOL', 'DATASAIDA', 'DATAINITRAB',
       'DATATERTRAB', 'TP', 'TD', 'TE', 'EQUIPE', 'ANO', 'MES', 'EUSD',
       'EUSD_FIO_B', 'LATITUDE', 'LONGITUDE', 'TIPSERV', 'DATA_VENC',
       'CODSERV'],
      dtype='object')

In [11]:
serv = pd.concat([tec, com], ignore_index=True).to_clipboard(
    index=False, decimal=",", sep=";"
)


In [ ]:
serv = pd.concat([tec, com], ignore_index=True).drop_duplicates(subset=["NUMOS"])


In [ ]:
serv.shape


(241420, 25)

In [ ]:
equipes.to_clipboard(index=False, decimal=",", sep=";")


In [12]:
df.groupby(["numos"]).size().reset_index(name="counts").sort_values(
    by="counts", ascending=False
).shape


(151575, 2)

In [ ]:
pd.read_parquet(
    "/Rotas-Inteligentes/results_v3/atribuicoes_2023-01-01.parquet"
).to_clipboard(index=False, decimal=",", sep=";")


In [ ]:
equipes["BASE_LON"] = -63.885464691387746
equipes["BASE_LAT"] = -8.738508095069408


In [ ]:
equipes.to_parquet("/Rotas-Inteligentes/data/equipes.parquet", index=False)


In [ ]:
equipes.head()


,TIP_EQUIPE,EQUIPE,DT_REF,DTHAPS_INI,DTHAPS_FIM,HH_TRAB,DATA_INICIO_TURNO,DATA_FIM_TURNO,HORA_BRUTA,HH_TOTAL,DTHAPS_FIM_AJUSTADO,HH_TRAB_AJUSTADO,DTHPAUSA_FIM,DTHPAUSA_INI
23649,I,PVLSN81,2023-01-01,2023-01-01 05:59:02,2023-01-01 14:51:30,8.87,2023-01-01 06:00:00,2023-01-01 15:00:00,8.0,532.47,2023-01-01 14:51:30,8.87,2023-01-01 11:25:08.000000000,2023-01-01 10:25:08.000000000
30547,I,PVOSN67,2023-01-01,2023-01-01 06:00:08,2023-01-01 15:01:27,9.02,2023-01-01 06:00:00,2023-01-01 15:00:00,8.0,541.32,2023-01-01 15:01:27,9.02,2023-01-01 11:30:44.000000000,2023-01-01 10:30:44.000000000
24075,I,PVLSN84,2023-01-01,2023-01-01 06:00:15,2023-01-01 14:56:07,8.93,2023-01-01 06:00:00,2023-01-01 15:00:00,8.0,535.87,2023-01-01 14:56:07,8.93,2023-01-01 11:28:09.000000000,2023-01-01 10:28:09.000000000
30421,I,PVOSN66,2023-01-01,2023-01-01 06:00:31,2023-01-01 15:10:57,9.17,2023-01-01 06:00:00,2023-01-01 15:00:00,8.0,550.43,2023-01-01 15:10:57,9.17,2023-01-01 11:35:36.999999999,2023-01-01 10:35:36.999999999
23406,I,PVLSN80,2023-01-01,2023-01-01 08:00:59,2023-01-01 18:20:18,10.32,2023-01-01 08:00:00,2023-01-01 18:00:00,8.0,619.32,2023-01-01 18:20:18,10.32,2023-01-01 14:10:35.000000000,2023-01-01 13:10:35.000000000


In [ ]:
Atendimentos = pd.merge(
    df[]


,tipo_serv,numos,datasol,dataven,datater_trab,TD,TE,equipe,dthaps_ini,dthaps_fim_ajustado,...,codserv,data_sol,data_venc,num_cdc,datasaida,datainitrab,datatertrab,job_id_vroom,distancia_vroom,duracao_vroom
0,técnico,20230000000001,2023-01-01 00:22:00,NaT,2023-06-06 14:08:14,17.13,44.00,PVLSN81,2023-01-01 05:59:02,2023-01-01 14:51:30,...,NaN,NaT,NaT,<NA>,NaT,NaT,NaT,20230000000001,None,12161.0
1,técnico,20230000000006,2023-01-01 01:32:52,NaT,2023-01-01 04:55:34,20.65,27.00,PVLSN81,2023-01-01 05:59:02,2023-01-01 14:51:30,...,NaN,NaT,NaT,<NA>,NaT,NaT,NaT,20230000000006,None,12161.0
2,técnico,20230000000011,2023-01-01 01:49:54,NaT,2023-01-01 06:58:19,17.25,27.32,PVLSN81,2023-01-01 05:59:02,2023-01-01 14:51:30,...,NaN,NaT,NaT,<NA>,NaT,NaT,NaT,20230000000011,None,12161.0
3,técnico,20230000000013,2023-01-01 01:53:46,NaT,2023-01-01 09:59:53,42.55,64.37,PVLSN81,2023-01-01 05:59:02,2023-01-01 14:51:30,...,NaN,NaT,NaT,<NA>,NaT,NaT,NaT,20230000000013,None,12161.0
4,técnico,20230000000015,2023-01-01 02:19:30,NaT,2023-01-01 03:53:27,17.05,35.90,PVLSN81,2023-01-01 05:59:02,2023-01-01 14:51:30,...,NaN,NaT,NaT,<NA>,NaT,NaT,NaT,20230000000015,None,12161.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,técnico,20240000407260,2024-12-31 11:31:00,NaT,2024-12-31 22:52:17,31.40,44.00,PVLPL46,2024-12-31 21:03:17,2025-01-01 06:07:27,...,NaN,NaT,NaT,<NA>,NaT,NaT,NaT,20240000407260,None,6436.0
225,comercial,120631516,2024-12-31 19:23:12,2025-01-04 08:00:00,2025-01-02 09:54:15,17.73,2.75,PVLPL46,2024-12-31 21:03:17,2025-01-01 06:07:27,...,3.0,2024-12-31 19:23:12,2025-01-04 08:00:00,2231016,2025-01-02 09:33:46,2025-01-02 09:51:30,2025-01-02 09:54:15,120631516,None,6436.0
226,comercial,120619518,2024-12-31 15:11:39,2025-02-03 23:59:59,2024-12-31 15:52:23,0.10,38.65,PVLPL46,2024-12-31 21:03:17,2025-01-01 06:07:27,...,83.0,2024-12-31 15:11:39,2025-02-03 23:59:59,1246276,2024-12-31 15:13:38,2024-12-31 15:13:44,2024-12-31 15:52:23,120619518,None,6436.0
227,técnico,20240000407681,2024-12-31 20:22:17,NaT,2025-01-02 10:03:16,32.97,28.85,PVLPL46,2024-12-31 21:03:17,2025-01-01 06:07:27,...,NaN,NaT,NaT,<NA>,NaT,NaT,NaT,20240000407681,None,6436.0


In [ ]:
serv["tipo_serv"] = serv["TIPSERV"].map({"T": "técnico", "C": "comercial"})


In [ ]:
serv.columns = serv.columns.str.lower()


In [ ]:
atendimento = pd.merge(df[["numos", "tipo_serv"]], serv[["numos", "tipo_serv"]], on=["numos"], how="outer", indicator=True)


In [ ]:
atendimento._merge.value_counts()


_merge
both          152226
right_only     89845
left_only          0
Name: count, dtype: int64

In [ ]:
df = pd.DataFrame(df)


In [ ]:
df.to_clipboard(index=False, decimal=",", sep=";")


In [32]:
BASE_LON = -63.885464691387746
BASE_LAT = -8.738508095069408


In [ ]:
baseTec = pd.read_parquet("E:/Rotas-Inteligentes/data/BaseTec.parquet")
baseCom = pd.read_parquet("E:/Rotas-Inteligentes/data/ServCom.parquet")


In [ ]:
baseCom.shape


(253958, 18)

In [ ]:
baseTec.shape


(65367, 11)

In [ ]:
v4 = pd.read_parquet(r"E:\Rotas-Inteligentes\results_v4\atribuicoes_2023-01-01.parquet")


In [ ]:
v3 = pd.read_parquet(r"E:\Rotas-Inteligentes\results_v3\atribuicoes_2023-01-01.parquet")


In [ ]:
display(v3.shape, v4.shape)


(52, 29)

(52, 27)

In [ ]:
v3.columns


Index(['tipo_serv', 'numos', 'datasol', 'dataven', 'datater_trab', 'TD', 'TE',
       'equipe', 'dthaps_ini', 'dthaps_fim_ajustado', 'inicio_turno',
       'fim_turno', 'dthpausa_ini', 'dthpausa_fim', 'dth_chegada_estimada',
       'dth_final_estimada', 'fim_turno_estimado', 'eta_source', 'base_lon',
       'base_lat', 'chegada_base', 'latitude', 'longitude', 'dt_ref', 'EUSD',
       'EUSD_FIO_B', 'job_id_vroom', 'distancia_vroom', 'duracao_vroom'],
      dtype='object')

In [ ]:
v4[['equipe', 'dthaps_ini', 'dthaps_fim_ajustado', 'inicio_turno',
       'fim_turno', 'dthpausa_ini', 'dthpausa_fim', 'dth_chegada_estimada',
       'dth_final_estimada', 'fim_turno_estimado', 'chegada_base', 'EUSD']]


,equipe,dthaps_ini,dthaps_fim_ajustado,inicio_turno,fim_turno,dthpausa_ini,dthpausa_fim,dth_chegada_estimada,dth_final_estimada,fim_turno_estimado,chegada_base,EUSD
0,PVLSN81,NaT,NaT,NaT,NaT,NaT,NaT,2023-01-01 11:50:43,2023-01-01 12:34:43.000,NaT,NaT,71.78
1,PVLSN81,NaT,NaT,NaT,NaT,NaT,NaT,2023-01-01 10:56:18,2023-01-01 11:23:18.000,NaT,NaT,54.29
2,PVLSN81,NaT,NaT,NaT,NaT,NaT,NaT,2023-01-01 09:51:46,2023-01-01 10:19:05.200,NaT,NaT,57.89
3,PVLSN81,NaT,NaT,NaT,NaT,NaT,NaT,2023-01-01 07:26:28,2023-01-01 08:30:50.200,NaT,NaT,125.87
4,PVLSN81,NaT,NaT,NaT,NaT,NaT,NaT,2023-01-01 12:41:09,2023-01-01 13:17:03.000,NaT,NaT,162.07
5,PVLSN81,NaT,NaT,NaT,NaT,NaT,NaT,2023-01-01 11:35:32,2023-01-01 11:49:24.800,NaT,NaT,100.70
6,PVLSN81,NaT,NaT,NaT,NaT,NaT,NaT,2023-01-01 10:26:04,2023-01-01 10:52:50.200,NaT,NaT,119.16
7,PVLSN22,NaT,NaT,NaT,NaT,NaT,NaT,2023-01-01 15:45:03,2023-01-01 15:58:03.000,NaT,NaT,7090.40
8,PVLSN22,NaT,NaT,NaT,NaT,NaT,NaT,2023-01-01 16:10:28,2023-01-01 16:45:16.000,NaT,NaT,218.64
9,PVLSN22,NaT,NaT,NaT,NaT,NaT,NaT,2023-01-01 16:56:31,2023-01-01 18:02:01.000,NaT,NaT,12.63
